<a href="https://colab.research.google.com/github/lucabem/CompeticionMineria/blob/main/modelo_80_f1_luis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

In [ ]:
dataset_url = 'https://github.com/lucabem/CompeticionMineria/blob/main/data/dataset_images.zip?raw=true'
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='train_data', 
                                   extract=True)

81059840/81057027 [==============================] - 1s 0us/step


In [ ]:
!rm -rf /root/.keras/datasets/train_data.tar.gz /root/.keras/datasets/train_data
!ls -l /root/.keras/datasets/

total 76
drwxr-xr-x 28 root root  4096 Mar  5 17:25 ImagesTrain
drwxr-xr-x  2 root root 73728 Mar  5 17:25 Test


In [ ]:
train_path = pathlib.Path('/root/.keras/datasets/ImagesTrain')
image_count = len(list(train_path.glob('*/*.jpg')))
print(image_count)

2600


In [ ]:
for direct in train_path.iterdir():
  if direct.is_dir():
    direct.rename(train_path / direct.name.split(sep='_100')[0])

In [ ]:
batch_size = 32
img_height = 224
img_width  = 224

In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 706kB 6.3MB/s 


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow_addons as tfa

In [ ]:
from keras.preprocessing.image import ImageDataGenerator  
from keras.applications import densenet  
from keras.models import Sequential, Model, load_model  
from keras.layers import Conv2D, MaxPooling2D  
from keras.layers import Activation, Dropout, Flatten, Dense  
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback  
from keras import regularizers  
from keras import backend as K  
import tensorflow as tf
keras = tf.keras

In [ ]:
batch_size  = 32
img_height = 224
img_width  = 224
train_path = pathlib.Path('/root/.keras/datasets/ImagesTrain')

In [ ]:
import shutil
import os

In [ ]:
train_ds_gen = ImageDataGenerator(vertical_flip    = True,
                                  horizontal_flip  = True,
                                  validation_split = 0.2)

In [ ]:
train_data_gen = train_ds_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_path,
                                                  shuffle=True,
                                                  target_size=(img_height, img_width),
                                                  subset='training',
                                                  class_mode='categorical')

Found 2080 images belonging to 26 classes.


In [ ]:
valid_data_gen = train_ds_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_path,
                                                  shuffle=True,
                                                  target_size=(img_height, img_width),
                                                  subset='validation',
                                                  class_mode='categorical')

Found 520 images belonging to 26 classes.


In [ ]:
test_data_gen = train_ds_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_path,
                                                  shuffle=True,
                                                  target_size=(img_height, img_width),
                                                  class_mode='categorical')

Found 2600 images belonging to 26 classes.


In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape = (224, 224, 3), 
                                            classes     = 26,
                                            include_top  = False)

image_batch, label_batch = next(iter(train_data_gen))
feature_batch = base_model(image_batch)
base_model.trainable = False



global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(26, activation='softmax')


model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])


94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
opt = tf.keras.optimizers.Adagrad()

callback = tf.keras.callbacks.EarlyStopping(monitor='f1_score', patience=5)

model.compile(loss="categorical_crossentropy",
              optimizer=opt,
	            metrics=[tfa.metrics.F1Score(num_classes=26, average='weighted')],
              )

In [ ]:
tf.random.set_seed(2021)

model_history = model.fit(  
    train_data_gen,
    epochs=24,
    validation_data=valid_data_gen,
    validation_steps= valid_data_gen.n // batch_size)

Epoch 1/24
65/65 [==============================] - 395s 6s/step - loss: 3.2970 - f1_score: 0.0572 - val_loss: 2.5713 - val_f1_score: 0.2226
Epoch 2/24
65/65 [==============================] - 392s 6s/step - loss: 2.5174 - f1_score: 0.2752 - val_loss: 2.2062 - val_f1_score: 0.4295
Epoch 3/24
65/65 [==============================] - 388s 6s/step - loss: 2.1463 - f1_score: 0.4607 - val_loss: 1.9527 - val_f1_score: 0.5470
Epoch 4/24
65/65 [==============================] - 386s 6s/step - loss: 1.9239 - f1_score: 0.5713 - val_loss: 1.7808 - val_f1_score: 0.6114
Epoch 5/24
65/65 [==============================] - 388s 6s/step - loss: 1.7365 - f1_score: 0.6464 - val_loss: 1.6610 - val_f1_score: 0.6610
Epoch 6/24
65/65 [==============================] - 387s 6s/step - loss: 1.6339 - f1_score: 0.6694 - val_loss: 1.5447 - val_f1_score: 0.7086
Epoch 7/24
65/65 [==============================] - 386s 6s/step - loss: 1.5335 - f1_score: 0.7237 - val_loss: 1.4575 - val_f1_score: 0.7343
Epoch 8/24
65

In [ ]:
! ls /root/.keras/datasets/

ImagesTrain  Test


In [ ]:
batch_size = 32
img_height = 224
img_width  = 224
test_path  = pathlib.Path('/root/.keras/datasets/Test')

In [ ]:
# predicting images
from keras.preprocessing import image
import numpy as np
import pandas as pd
from google.colab import files

def make_predictions(model_final,
                     path_test ='/root/.keras/datasets/Test',
                     name_file_submission = 'submission.csv'):

  images_test = os.listdir(path_test)
  samples_to_predict = []

  for img in images_test:
    path_img = os.path.join(path_test, img)
    img = image.load_img(path_img,
                        target_size=(img_width, img_height))
    x = image.img_to_array(img)
    samples_to_predict.append(x)

  samples_to_predict  = np.array(samples_to_predict)
  predictions = model_final.predict(samples_to_predict)
  classes = np.argmax(predictions, axis = 1)

  data = {'id.jpg': [img for img in images_test], 'label': classes}
  data = pd.DataFrame(data)
  data.to_csv(name_file_submission, index = False)

  return data

In [ ]:
make_predictions(model)

,id.jpg,label
0,Forest_DEB_100.0_(-3.7729241933020035_-77.0582...,11
1,Forest_CDB_88.03655505180359_(-13.934666687262...,5
2,Close_Shrubland_50_90.78353643417358_(-17.8786...,2
3,Cereal_Cropland_100.0_(52.468799114853_52.2072...,0
4,Forest_CEN_84.61316227912903_(29.5696645443054...,8
...,...,...
515,Forest_OEB_88.33755254745483_(-12.425497009941...,15
516,Forest_DDB_99.86563920974731_(41.4619216015933...,4
517,Marine_Water_100.0_(71.54470383618786_179.4201...,19
518,Forest_OEN_86.29198670387268_(56.8554523102654...,16


In [ ]:
while True:pass

KeyboardInterrupt: ignored